## Description of the dataset

| Column header | Unit of measure | Description |
|---------------|-----------------|-------------|
| n            | ms             | Timestamp of the recorded gaze sample since the beginning of the recording |
| x            | dva            | $\theta_h$ for the cyclopean eye |
| y            | dva            | $\theta_v$ for the cyclopean eye |
| lx           | dva            | $\theta_h$ for the left eye |
| ly           | dva            | $\theta_v$ for the left eye |
| rx           | dva            | $\theta_h$ for the right eye |
| ry           | dva            | $\theta_v$ for the right eye |
| xT*          | dva            | $\theta_h$ for the stimulus, relative to the cyclopean eye |
| yT*          | dva            | $\theta_v$ for the stimulus, relative to the cyclopean eye |
| zT           | m              | Depth of the stimulus |
| clx          | m              | X position of the center of the left eyeball, relative to the camera origin |
| cly          | m              | Y position of the center of the left eyeball, relative to the camera origin |
| clz          | m              | Z position of the center of the left eyeball, relative to the camera origin |
| crx          | m              | X position of the center of the right eyeball, relative to the camera origin |
| cry          | m              | Y position of the center of the right eyeball, relative to the camera origin |
| crz          | m              | Z position of the center of the right eyeball, relative to the camera origin |
| round        |                | recording round (1-3) |
| participant  |                | participant ID (001-465) |
| session      |                | recording session (1-2) |
| task         |                | task category (1-5) |

In [1]:
# Dataset parameters
SAMPLE_NUM = 40 # Number of participants to downsample to
NUM_OF_MODELS = 10  # Number of models to build based on randomly selected participants
TEST_RATIO = 0.2 # Ratio of test data to total data

# Sliding window parameters (data sampling @250 Hz)
WINDOW_SIZE = 10 * 250 # 10 seconds of data
WINDOW_STRIDE = 250 # 1 second stride

# Hyperparameters
HIDDEN_SIZE = 128
NUM_LAYERS = 2
NUM_EPOCHS = 20
LEARNING_RATE = 0.001

# Output parameters
SIGMOID_THRESHOLD = 0.8

In [2]:
import pandas as pd

# Load the dataset using Dask with specified dtypes, filter task PUR
df = pd.read_parquet('dataset/gazebasevr.parquet', filters=[('task', '=', 2)])

# Exclude metadata columns
df = df.drop(columns=['round', 'session', 'task'])

# Exclude unfeasible features
df = df.drop(columns=['xT', 'yT', 'zT'])

# Print memory usage
print(f"{df.memory_usage().sum() / 1024 ** 2:.2f} MB")

2287.56 MB


In [3]:
import numpy as np

# Randomly sample 20 participants
participants = np.random.choice(df["participant"].unique(), SAMPLE_NUM, replace=False)

# Print selected participants
print(f"Selected participants: {participants}")

# Filter the dataset to include only the selected participants
df = df[df["participant"].isin(participants)]

# Print memory usage
print(f"{df.memory_usage().sum() / 1024 ** 2:.2f} MB")

Selected participants: [272  95 245 105 155 336 414 265 271 170 352 442  65 158  85   2 175 162
 229 250 209 370 115 415 388 407 171 219 354 317  58 381 166 429 116 330
 301 355 262  10]
259.16 MB


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

def evaluate_model(y_true, y_pred):
    tp = sum((y_true == 1) & (y_pred == 1))
    tn = sum((y_true == 0) & (y_pred == 0))
    fp = sum((y_true == 0) & (y_pred == 1))
    fn = sum((y_true == 1) & (y_pred == 0))
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall)

    return {
        "f1": f1,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
    }


def split_train_test(df, test_size):
    # Ensure the dataframe is sorted by timestamp
    df = df.sort_values(by="n")

    # Split the data for each participant
    train_dfs = []
    test_dfs = []

    for participant_id, group in df.groupby("participant"):
        train, test = train_test_split(group, test_size=test_size, shuffle=False)
        train_dfs.append(train)
        test_dfs.append(test)

    train_df = pd.concat(train_dfs).reset_index(drop=True)
    test_df = pd.concat(test_dfs).reset_index(drop=True)

    return train_df, test_df


def create_sliding_windows(df, window_size, stride, participant_id):
    X = []
    y = []

    for pid, group in df.groupby("participant"):
        group = group.sort_values(by="n").reset_index(drop=True)
        features = group.drop(columns=["participant", "n"]).values
        label = int(pid == participant_id)

        for start in range(0, len(features) - window_size + 1, stride):
            window = features[start : start + window_size]
            X.append(window)
            y.append(label)

    return np.array(X), np.array(y)


def prepare_participant_data(
    train_df, test_df, participant_id, window_size, stride
):
    X_train, y_train = create_sliding_windows(
        train_df, window_size, stride, participant_id
    )
    X_test, y_test = create_sliding_windows(
        test_df, window_size, stride, participant_id
    )
    return X_train, X_test, y_train, y_test

In [5]:
# Fill missing values with forward fill and backward fill
df = df.ffill().bfill()

In [6]:
# Split the data
train_df, test_df = split_train_test(df, TEST_RATIO)

In [7]:
# Print the number of samples in the training and test set
total_samples = len(train_df) + len(test_df)
print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"Ratio: train {len(train_df) / total_samples:.2f}, test {len(test_df) / total_samples:.2f}")

Training samples: 3293904
Test samples: 823492
Ratio: train 0.80, test 0.20


In [8]:
train_df.head()

,n,x,y,lx,ly,rx,ry,clx,cly,clz,crx,cry,crz,participant
0,0.0000,2.9118,-4.0018,6.0274,-3.4764,-0.4068,-4.5371,-0.0323,0.0029,-0.0293,0.0327,0.0040,-0.0297,2
1,0.0000,0.0014,-5.7918,2.7773,-5.4001,-2.5670,-6.5306,-0.0331,0.0058,-0.0297,0.0326,0.0078,-0.0295,2
2,0.0000,-0.7793,-0.3799,2.2850,-0.6278,-4.0353,-0.1743,-0.0331,0.0058,-0.0287,0.0327,0.0076,-0.0291,2
3,0.0000,0.8193,-4.3130,4.0541,-4.5366,-2.4185,-4.0864,-0.0323,0.0016,-0.0281,0.0324,0.0032,-0.0293,2
4,3.7914,2.9004,-3.9961,5.9637,-3.5508,-0.4068,-4.6690,-0.0323,0.0029,-0.0293,0.0327,0.0041,-0.0297,2


In [9]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import numpy as np


# Custom Dataset class
class GazeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = np.array(data)
        self.labels = np.array(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(
            self.labels[idx], dtype=torch.float32
        )


# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [10]:
import IPython
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import os

participants = np.random.choice(
    df["participant"].unique(), NUM_OF_MODELS, replace=False
)
# participants = [201, 306, 169]
print(f"Models for participants: {participants}")

input_size = train_df.shape[1] - 2  # Exclude participant and n columns

results = []

for participant_id in participants:
    model_path = f"models/rnn-lstm/{participant_id}.pth"

    if not os.path.exists("models/rnn-lstm"):
        os.makedirs("models/rnn-lstm", exist_ok=True)

    if os.path.exists(model_path):
        print(f"Loading model for participant {participant_id} from disk")
        model = LSTMClassifier(input_size, HIDDEN_SIZE, NUM_LAYERS).to(device)
        model.load_state_dict(torch.load(model_path))
    else:
        print(f"Training model for participant {participant_id}")
        X_train, X_test, y_train, y_test = prepare_participant_data(
            train_df, test_df, participant_id, WINDOW_SIZE, WINDOW_STRIDE
        )

        train_dataset = GazeDataset(X_train, y_train)
        test_dataset = GazeDataset(X_test, y_test)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Model, loss function, optimizer
        model = LSTMClassifier(input_size, HIDDEN_SIZE, NUM_LAYERS).to(device)

        weight_ratio = (len(y_train) - sum(y_train)) / sum(y_train)
        pos_weight = torch.tensor([weight_ratio]).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # Training loop
        for epoch in range(NUM_EPOCHS):
            model.train()
            for data, labels in tqdm(
                train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=False
            ):
                data, labels = data.to(device), labels.to(device)
                outputs = model(data)
                loss = criterion(outputs.squeeze(1), labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            IPython.display.clear_output()
            print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}")

        # Save the model
        torch.save(model.state_dict(), f"models/rnn-lstm/{participant_id}.pth")

    # Evaluation
    model.eval()
    y_true = np.array([])
    y_pred = np.array([])
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            predicted = (torch.sigmoid(outputs) > SIGMOID_THRESHOLD).long().squeeze()
            y_true = np.concatenate((y_true, labels.cpu().numpy()))
            y_pred = np.concatenate((y_pred, predicted.cpu().numpy()))

    result = evaluate_model(y_true, y_pred)
    print(result)
    results.append(result)

# Save results
if not os.path.exists("results"):
    os.makedirs("results", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv("results/rnn-lstm.csv", index=False)

Epoch [20/20], Loss: 0.0972
{'f1': 0.842857142857143, 'accuracy': 0.9849004804392587, 'precision': 0.7515923566878981, 'recall': 0.959349593495935, 'tp': np.int64(118), 'tn': np.int64(2752), 'fp': np.int64(39), 'fn': np.int64(5)}
